In [ ]:
%cd ~
import pyfits,glob,time,scipy,sys
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from IPython import parallel
from scipy.optimize import fmin_powell

topdir='/Users/cslage/Research/LSST/code/'
thedir=topdir+'phosim-walter/output/'
#thedir=topdir+'phosim-d912a460f878/output/'
#thedir=topdir+'phosim-d912_orig/output/'
%cd $thedir

configfile=topdir+'GUI/sextractor/default-array_dither.sex'
paramfile=topdir+'GUI/sextractor/default-array_dither.param'
maskdir=topdir+'GUI/sextractor/masks/'
sys.path.append(topdir+'GUI/profiles/')

#%matplotlib inline

In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
# This definition runs sextractor in parallel and can be given to IPython's parallel map function along with
#   the file list
def ovsub_runsex_makereg(fitsfilename):
    import pyfits
    import numpy as np
    from subprocess import call
    topdir='/Users/cslage/Research/LSST/code/GUI/'
    thedir=topdir+'profiles'
    %cd $thedir

    configfile=topdir+'sextractor/default-array_dither.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'
    maskdir=topdir+'sextractor/masks/'

    def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
        overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
        image=image[:y_overscan_start,:x_overscan_start]
        finalimg=image-np.matrix(np.median(overscan,axis=1)).T*np.ones(np.shape(image)[1])
        return finalimg

    def make_reg_from_ldac(cat_ldac_file,text_tag):
        thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
        f = open(cat_ldac_file+'.reg','w')
        for i in range(len(thecat)):
            xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
            r=thecat['A_IMAGE'][i]
            thetext=thecat[text_tag][i]
            f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
        f.close()
    for i in range(1,17):
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname)
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022)   # cut off the overscan
        outname=fitsfilename[:-5]+extname+'.fits'
        pyfits.writeto(outname,overscansubimg,clobber=True)
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat'])
        make_reg_from_ldac(outname+'.cat','NUMBER')

In [ ]:
#zfilelist=sort(glob.glob(thedir+'lsst_e_100000_f2_R22_S11_E???.fits')+glob.glob(thedir+'lsst_e_*_f2_R22_S11_E000.fits'))
zfilelist=sort(glob.glob(thedir+'bf/lsst_e_*_f2_R22_S11_E000.fits'))

zfilelist

In [ ]:
for fitsfilename in zfilelist: 
    tfile1=time.time() 
    tstart=time.time() 
    outname=fitsfilename[:-5]+'_SEX.fits' 
    test=call(["sex",fitsfilename,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
    make_reg_from_ldac(outname+'.cat','NUMBER') 
    tend=time.time() 
    print "Time taken for file "+str(fitsfilename[-24:-19])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny,nstamps):
        # This packages up a set of nstamps postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=nstamps

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.data=zeros([nx,ny,nstamps])
        self.xoffset=zeros([nstamps])
        self.yoffset=zeros([nstamps])
        self.imax=zeros([nstamps])


def BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    xcoomin = 1990
    xcoomax = 2400
    ycoomin = 2000
    ycoomax = 2450
    spotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,numspots)
    img=pyfits.getdata(fitsfilename,0) 
    catname=fitsfilename[:-5]+'_SEX.fits.cat.reg' 
    catfile = open(catname,'r')
    catlines = catfile.readlines()
    catfile.close()
    n=0
    for line in catlines:
        try:
            size = float(line.split()[3].split('#')[0])
            if size < minsize or size > maxsize:
                continue
            xcoord = float(line.split()[1])
            ycoord = float(line.split()[2])
            if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                continue
            xint = int(xcoord-0.5)
            yint = int(ycoord-0.5)
            xmin = xint - int(nx/2)
            xmax = xint + int(nx/2) + 1
            ymin = yint - int(ny/2)
            ymax = yint + int(ny/2) + 1
            stamp = img[ymin:ymax, xmin:xmax]
           
            xsum = 0.0
            ysum = 0.0
            datasum = 0.0
             
            for i in range(nx):
                for j in range(ny):
                    spotlist.data[i,j,n] = float(stamp[j,i])                    
                    ysum += spotlist.y[j] * spotlist.data[i,j,n]
                    xsum += spotlist.x[i] * spotlist.data[i,j,n]
                    datasum += spotlist.data[i,j,n]
            xoff = xsum / datasum
            yoff = ysum / datasum
            spotlist.xoffset[n] = xcoord - xint - 1.0
            spotlist.yoffset[n] = ycoord - yint - 1.0     
                    
            n += 1
            if n == numspots:
                return spotlist
        except:
            continue
    # Reaching this point means we found less spots than requested.
    newspotlist = Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny,n)
    newspotlist.xoffset = spotlist.xoffset[0:n]
    newspotlist.yoffset = spotlist.yoffset[0:n]
    newspotlist.data = spotlist.data[:,:,0:n]
    del spotlist
    return newspotlist

import forward

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                #if spot == 78 and ii == 4 and jj == 4 or spot==0:
                    #print "ii = %d, jj = %d, img = %.4f"%(ii,jj,spotlist.data[ii,jj,spot])
                #print "ii = %d, jj = %d,xl = %.2f, xh = %.2f, yl = %.2f, yh = %.2f"%(ii,jj,xl,xh,yl,yh)
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    #print "Imax = %.1f, sigmax = %.2f, sigmay = %.2f, fom = %.1f"%(Imax, sigmax, sigmay, fom)
    return fom


In [ ]:
from scipy import stats
nx = 9
ny = 9
numspots = 300
segmentnumber = 7

fitsfilename = zfilelist[20]
print fitsfilename
    
spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny, 0.9, 1.4)

print "nstamps = %d"%spotlist.nstamps

sys.exit()
param0 = [1.00, 1.00]

starttime = time.time()
fom = FOM(param0)
elapsed = time.time() - starttime
Spotsum = []
print "CFom = %.1f, time = %.3f seconds"%(fom, elapsed)
for spot in range(numspots):
    Spotsum.append(spotlist.data[:,:,spot].sum())
Spotsum = array(Spotsum)
slope, intercept, r_value, p_value, std_err = stats.linregress(Spotsum,spotlist.imax)
xplot=linspace(Spotsum.min(),Spotsum.max(),100)
yplot = slope * xplot + intercept

figure()
text(Spotsum.mean(), Spotsum.min() * 1.00,"R^2 = %.4f"%r_value)
text(Spotsum.mean(), Spotsum.min() * 0.95,"Std_error = %.4f"%std_err)
scatter(Spotsum,spotlist.imax)
plot(xplot, yplot, color='red')
xlabel("Spotlist sum")
ylabel("Modeled Imax")
savefig("Forward_Model_Imax_SpotSum.png")

In [ ]:
nx = 9
ny = 9
numspots = 300
results = []
spotlists=[]
segmentnumber = 0

for fitsfilename in zfilelist:
    print fitsfilename
    param0 = [1.0, 1.0]
    
    spotlist = BuildSpotList(fitsfilename, segmentnumber, numspots, nx, ny,0.2,2.0)
    print "nstamps = %d"%spotlist.nstamps

    spotlists.append(spotlist)
    args = ()#(spotlist)
    Result = fmin_powell(FOM, param0, args)
    print fitsfilename
    print Result
    
    results.append(Result)


In [ ]:
print zfilelist[-9]

In [ ]:
from scipy import stats
figure()
title("Brighter-Fatter Effect on PhoSim Generated Spots")
imaxs = []
sigmaxs = []
sigmays = []
gain = 1.0
for i,result in enumerate(results):
    imax = spotlists[i].imax.mean()
    ADU_correction = Area(-0.5,0.5,-0.5,0.5,result[0],result[1],1.0)
    imaxs.append(imax * ADU_correction * gain)
    sigmaxs.append(result[0])
    sigmays.append(result[1])
    #if result[2] < 1.06:
    #    print "I = %d, Imax = %.1f, sigmax = %.3f, sigmay = %.3f"%(i,result[0],result[1],result[2])
scatter(imaxs, sigmaxs, color = 'green', lw = 2, label = 'Sigma-x')
scatter(imaxs, sigmays, color = 'red', lw = 2, label = 'Sigma-y')

slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs[-10:-1],sigmaxs[-10:-1])
#xlist = imaxs[3:6]+imaxs[8:10]+imaxs[12:16]+imaxs[18:-1]
#ylistx = sigmaxs[3:6]+sigmaxs[8:10]+sigmaxs[12:16]+sigmaxs[18:-1]
#ylisty = sigmays[3:6]+sigmays[8:10]+sigmays[12:16]+sigmays[18:-1]

#slope, intercept, r_value, p_value, std_err = stats.linregress(xlist,ylistx)
#slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs,sigmaxs)

xplot=linspace(-5000.0,150000.0,100)
yplot = slope * xplot + intercept
plot(xplot, yplot, color='blue', lw = 2, ls = '--')
tslope = slope/intercept * 100.0 * 50000.0
text(5000,1.24,"X Slope = %.2f percent per 50,000 electrons"%tslope)

slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs[-10:-1],sigmays[-10:-1])
#slope, intercept, r_value, p_value, std_err = stats.linregress(xlist,ylisty)
#slope, intercept, r_value, p_value, std_err = stats.linregress(imaxs,sigmays)

xplot=linspace(-5000.0,150000.0,100)
yplot = slope * xplot + intercept
plot(xplot, yplot, color='black', lw = 2, ls = '--')
tslope = slope/intercept * 100.0 * 50000.0
#text(110000,1.105,"chargesharing on")
#text(110000,1.10,"chargediffusion on")
text(5000,1.23,"Y Slope = %.2f percent per 50,000 electrons"%tslope)


#text(110000,1.046,"chargesharing on")
#text(110000,1.043,"chargediffusion on")

#text(60000,1.031,"chargesharing off")
#text(60000,1.028,"chargediffusion on")
#plot([0.0,150000], [intercept,intercept],color = 'black',lw = 2, ls = '--')
#text(60000,1.014,"chargesharing off")
#text(60000,1.011,"chargediffusion off")
#plot([0.0,150000], [1.0095,1.0095],color = 'black',lw = 2, ls = '--')

#text(1000,1.023,"m=23")
#text(120000,1.062,"m=19")

ylim(1.15,1.25)
#ylim(1.07, 1.14)
xlim(0,160000)
xlabel('Central Peak(electrons)')
ylabel('Sigma (Pixels)')
legend(loc= 'lower right')
savefig(thedir+"Forward_Model_Spots.png")

In [ ]:
figure()
testn = 0
spot = 100
spotlist = spotlists[testn]
Result = results[testn]
Imax = spotlist.imax[spot]
sigmax = Result[0]
sigmay = Result[1]
seqno = 0#int(zfilelist[testn].split("_")[3])
suptitle=("Sequence # %d, Spot # %d"%(seqno,spot))
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[spot] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[spot] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)

subplots_adjust(wspace = 1.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,spot],interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,spot], lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),spot], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
legend(loc = (-4.0,0.8))
savefig("Typical_Fit_Powell")


In [ ]:
print len(zfilelist)
print results[]

In [ ]:
from scipy import stats
figure()
exp_current=[]
intensities=[]
for i,fitsfilename in enumerate(zfilelist[0:93]):
    try:
        exptime=pyfits.getheader(fitsfilename,0)['EXPTIME'] 
        mondiode=pyfits.getheader(fitsfilename,0)['MONDIODE'] 
        exp_current.append(float(exptime) * mondiode / (1.0E-9))
        ADU_correction = Area(-0.5,0.5,-0.5,0.5,results[i][0],results[i][1],1.0)
        intensity = spotlists[i].imax.mean() * ADU_correction
        intensities.append(intensity/1000.0)
    except:
        continue
slope, intercept, r_value, p_value, std_err = stats.linregress(exp_current[0:30],intensities[0:30])

xplot=linspace(0.0,7.0,100)
yplot = slope * xplot + intercept
scatter(exp_current,intensities)
plot(xplot, yplot, color='red')
text(0.5,70,"Linear Fit R^2 = %.5f"%r_value)
xlabel("Exp Time * Monitor Diode (nA-sec)")
ylabel("Modeled Peak Intensity (kADU)")
#xlim(0.0, 7.0)
#ylim(0.0,350.0)
savefig("Intensity_Check_Powell.png")

In [ ]:
x=array([1,2,3,4,5])
print x
resize(x,(2))
print x

In [ ]:
rand()